## Taxi-V3 using SARSA made by Andrea Bolla - 4482930

For a learning agent in any Reinforcement Learning algorithm it’s policy can be of two types:

    On Policy: the learning agent learns the value function according to the current action derived from the policy currently being used 
    
    Off Policy: the learning agent learns the value function according to the action derived from another policy 

SARSA technique is an On Policy and uses the action performed by the current policy to learn the Q-value.

The equation for SARSA depends on the current state, current action, reward obtained, next state and next action. SARSA stands for State Action Reward State Action which symbolizes the tuple (s, a, r, s’, a’).

# Step 1: Importing the required libraries

In [1]:
import sys
sys.path.append('/Users/boez/opt/anaconda3/lib/python3.9/site-packages')

import gym
import numpy as np
import random

sys: is used for the path of my libraries (NB change it with yours)

gym: is a standard API for reinforcement learning, and has diverse collection of reference environments

numpy: is a Python library used for working with arrays

random:  is a module to implement pseudo-random number generators

# Step 2: Building the environment

In [2]:
env = gym.make("Taxi-v3", render_mode="human").env

env.reset()
env.render()

print("Action Space {}".format(env.action_space))

print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


env.reset: Resets the environment and returns a random initial state.

env.step(action): Step the environment by one timestep. Returns:
    
    observation: Observations of the environment

    reward: If your action was beneficial or not

    done: Indicates if we have successfully picked up and dropped off a passenger, also called one episode

    truncated: if episode truncates due to a time limit or a reason that is not defined as part of the task MDP.

    info: Additional info such as performance and latency for debugging purposes

env.render: Renders one frame of the environment from 0-5 where:

    0 = south
    1 = north
    2 = east
    3 = west
    4 = pickup
    5 = dropoff

The 500 states correspond to a encoding of the taxi's location, the passenger's location, and the destination location

In [3]:
env.P[123]

{0: [(1.0, 223, -1, False)],
 1: [(1.0, 23, -1, False)],
 2: [(1.0, 123, -1, False)],
 3: [(1.0, 103, -1, False)],
 4: [(1.0, 123, -10, False)],
 5: [(1.0, 123, -10, False)]}

The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.

In this env, probability is always 1.0.

The nextstate is the state we would be in if we take the action at this index of the dict

All the movement actions have a -1 reward 
the pickup/dropoff actions have -10 reward in this particular state. 
If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action 

done is used to tell us when we have successfully dropped off a passenger in the right location. 

Each successfull dropoff is the end of an episode

# Step 3: Initializing different parameters

In [4]:
# Defining parameters
epsilon = 1 # Total exploration and no exploitation
alpha = 0.3
gamma = 0.95

# Training parameters
max_episodes = 100000  # number of episodes to use for training
max_steps = 100   # maximum number of steps per episode

#Initializing the Q-table 500x6
Q = np.zeros((env.observation_space.n, env.action_space.n))

The equation for SARSA depends on the current state, current action, reward obtained, next state and next action.

Q(s_t,a_t) = Q(s_t,a_t) + alpha* ( r_(t+1) + gamma* Q(s_(t+1),a_(t+1)) - Q(s_t,a_t) )

    
Where: 
     
    ϵ (epsilon) is the paramenter which choose between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). 
    
    α (alpha) is the learning rate, it is the extent to which our Q-values are being updated in every iteration.
    
    γ (gamma) is the discount factor determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, insted a discount factor of 0 makes our agent consider only immediate reward (greedy).

# Step 4: Defining functions for the learning process

In [5]:
# Function to choose the next action
def choose_action(state):

    action=0

    if np.random.uniform(0, 1) < epsilon:

        action = env.action_space.sample()   # explore

    else:
        
        action = np.argmax(Q[state, :])      # exploit
        
    return action

# Function to update the Q-value
def update(state, state2, reward, action, action2):
    
    predict = Q[state, action]
    target = reward + gamma * Q[state2, action2]
    
    Q[state, action] = Q[state, action] + alpha * (target - predict)

Choose_action() allow the agent to choose the next action, it all depens on the random number and the epsilon value, if epsilon is bigger than the random number the agent will choose to explore, otherwise to exploit and choose in the update Q table the best action.

Update() is used to update the Q table, following the SARSA equation, when the agent choose to explore.

# Step 5: Training the learning agent

In [6]:
# Initializing parameters
score = 0
goal = 0 
episode = 0


# Starting the SARSA learning
for episode in range(max_episodes):

    # Print each episode
    print("Episode: ", episode)
    print("Goal reached: ", goal)
    print("Score: ", score)
    print(" ")
    
    state1 = env.reset()
    state1 = state1[0]
    action1 = choose_action(state1)
    
    # Reset parameters
    s = 0
    score = 0
    done = False
    
    # Decreasing epsilon
    epsilon -= 0.005
    
    if epsilon < 0.05:
        epsilon = 0.05    # epsilon min: total exploitation (95%)
 

    for s in range(max_steps):
        
        # Rendering
        env.render()
         
        # Get the next state
        state2, reward, done, truncate, info = env.step(action1)
 
        # Choose the next action
        action2 = choose_action(state2)
         
        # Update the Q-value
        update(state1, state2, reward, action1, action2)
 
        state1 = state2
        action1 = action2
         
        # Updating parameters
        s += 1
        score += reward
        
        if reward==20:
            goal +=1
         
        #If at the end of learning process
        if done:
            break

Episode:  0
Goal reached:  0
Score:  0
 
Episode:  1
Goal reached:  0
Score:  -415
 
Episode:  2
Goal reached:  0
Score:  -433
 
Episode:  3
Goal reached:  0
Score:  -433
 
Episode:  4
Goal reached:  0
Score:  -397
 
Episode:  5
Goal reached:  0
Score:  -379
 
Episode:  6
Goal reached:  0
Score:  -442
 
Episode:  7
Goal reached:  0
Score:  -397
 
Episode:  8
Goal reached:  0
Score:  -361
 
Episode:  9
Goal reached:  0
Score:  -415
 
Episode:  10
Goal reached:  0
Score:  -424
 
Episode:  11
Goal reached:  0
Score:  -298
 
Episode:  12
Goal reached:  0
Score:  -352
 
Episode:  13
Goal reached:  0
Score:  -406
 
Episode:  14
Goal reached:  0
Score:  -361
 
Episode:  15
Goal reached:  0
Score:  -334
 
Episode:  16
Goal reached:  0
Score:  -298
 
Episode:  17
Goal reached:  0
Score:  -388
 
Episode:  18
Goal reached:  0
Score:  -388
 
Episode:  19
Goal reached:  0
Score:  -361
 
Episode:  20
Goal reached:  0
Score:  -370
 
Episode:  21
Goal reached:  0
Score:  -316
 
Episode:  22
Goal reach

Episode:  181
Goal reached:  11
Score:  -56
 
Episode:  182
Goal reached:  11
Score:  -109
 
Episode:  183
Goal reached:  12
Score:  -95
 
Episode:  184
Goal reached:  12
Score:  -136
 
Episode:  185
Goal reached:  13
Score:  -56
 
Episode:  186
Goal reached:  13
Score:  -136
 
Episode:  187
Goal reached:  14
Score:  -91
 
Episode:  188
Goal reached:  15
Score:  -93
 
Episode:  189
Goal reached:  15
Score:  -136
 
Episode:  190
Goal reached:  16
Score:  -91
 
Episode:  191
Goal reached:  17
Score:  3
 
Episode:  192
Goal reached:  18
Score:  -63
 
Episode:  193
Goal reached:  18
Score:  -118
 
Episode:  194
Goal reached:  18
Score:  -118
 
Episode:  195
Goal reached:  18
Score:  -100
 
Episode:  196
Goal reached:  19
Score:  -23
 
Episode:  197
Goal reached:  19
Score:  -109
 
Episode:  198
Goal reached:  20
Score:  -36
 
Episode:  199
Goal reached:  21
Score:  -11
 
Episode:  200
Goal reached:  21
Score:  -118
 
Episode:  201
Goal reached:  22
Score:  -50
 
Episode:  202
Goal reached:

Episode:  358
Goal reached:  141
Score:  2
 
Episode:  359
Goal reached:  142
Score:  -27
 
Episode:  360
Goal reached:  143
Score:  -25
 
Episode:  361
Goal reached:  144
Score:  -29
 
Episode:  362
Goal reached:  145
Score:  7
 
Episode:  363
Goal reached:  146
Score:  9
 
Episode:  364
Goal reached:  147
Score:  -7
 
Episode:  365
Goal reached:  148
Score:  -24
 
Episode:  366
Goal reached:  149
Score:  -10
 
Episode:  367
Goal reached:  150
Score:  12
 
Episode:  368
Goal reached:  151
Score:  -21
 
Episode:  369
Goal reached:  152
Score:  -14
 
Episode:  370
Goal reached:  153
Score:  -89
 
Episode:  371
Goal reached:  154
Score:  7
 
Episode:  372
Goal reached:  154
Score:  -109
 
Episode:  373
Goal reached:  155
Score:  8
 
Episode:  374
Goal reached:  156
Score:  -85
 
Episode:  375
Goal reached:  157
Score:  -2
 
Episode:  376
Goal reached:  158
Score:  12
 
Episode:  377
Goal reached:  159
Score:  -49
 
Episode:  378
Goal reached:  160
Score:  -35
 
Episode:  379
Goal reached

Episode:  535
Goal reached:  314
Score:  6
 
Episode:  536
Goal reached:  315
Score:  -13
 
Episode:  537
Goal reached:  316
Score:  5
 
Episode:  538
Goal reached:  317
Score:  4
 
Episode:  539
Goal reached:  318
Score:  -1
 
Episode:  540
Goal reached:  319
Score:  10
 
Episode:  541
Goal reached:  320
Score:  -20
 
Episode:  542
Goal reached:  321
Score:  6
 
Episode:  543
Goal reached:  322
Score:  -3
 
Episode:  544
Goal reached:  323
Score:  10
 
Episode:  545
Goal reached:  324
Score:  -27
 
Episode:  546
Goal reached:  325
Score:  -19
 
Episode:  547
Goal reached:  326
Score:  7
 
Episode:  548
Goal reached:  327
Score:  6
 
Episode:  549
Goal reached:  328
Score:  9
 
Episode:  550
Goal reached:  329
Score:  5
 
Episode:  551
Goal reached:  330
Score:  -13
 
Episode:  552
Goal reached:  331
Score:  5
 
Episode:  553
Goal reached:  332
Score:  -4
 
Episode:  554
Goal reached:  333
Score:  -7
 
Episode:  555
Goal reached:  334
Score:  13
 
Episode:  556
Goal reached:  335
Score

Episode:  714
Goal reached:  493
Score:  4
 
Episode:  715
Goal reached:  494
Score:  7
 
Episode:  716
Goal reached:  495
Score:  12
 
Episode:  717
Goal reached:  496
Score:  7
 
Episode:  718
Goal reached:  497
Score:  9
 
Episode:  719
Goal reached:  498
Score:  -23
 
Episode:  720
Goal reached:  499
Score:  -9
 
Episode:  721
Goal reached:  500
Score:  4
 
Episode:  722
Goal reached:  501
Score:  -9
 
Episode:  723
Goal reached:  502
Score:  1
 
Episode:  724
Goal reached:  503
Score:  8
 
Episode:  725
Goal reached:  504
Score:  -27
 
Episode:  726
Goal reached:  505
Score:  1
 
Episode:  727
Goal reached:  506
Score:  6
 
Episode:  728
Goal reached:  507
Score:  8
 
Episode:  729
Goal reached:  508
Score:  -3
 
Episode:  730
Goal reached:  509
Score:  6
 
Episode:  731
Goal reached:  510
Score:  -7
 
Episode:  732
Goal reached:  511
Score:  -6
 
Episode:  733
Goal reached:  512
Score:  4
 
Episode:  734
Goal reached:  513
Score:  13
 
Episode:  735
Goal reached:  514
Score:  10


...

Episode:  3503
Goal reached:  3282
Score:  5
 
Episode:  3504
Goal reached:  3283
Score:  4
 
Episode:  3505
Goal reached:  3284
Score:  6
 
Episode:  3506
Goal reached:  3285
Score:  4
 
Episode:  3507
Goal reached:  3286
Score:  4
 
Episode:  3508
Goal reached:  3287
Score:  7
 
Episode:  3509
Goal reached:  3288
Score:  -12
 
Episode:  3510
Goal reached:  3289
Score:  9
 
Episode:  3511
Goal reached:  3290
Score:  -14
 
Episode:  3512
Goal reached:  3291
Score:  6
 
Episode:  3513
Goal reached:  3292
Score:  10
 
Episode:  3514
Goal reached:  3293
Score:  -5
 
Episode:  3515
Goal reached:  3294
Score:  4
 
Episode:  3516
Goal reached:  3295
Score:  7
 
Episode:  3517
Goal reached:  3296
Score:  9
 
Episode:  3518
Goal reached:  3297
Score:  8
 
Episode:  3519
Goal reached:  3298
Score:  -4
 
Episode:  3520
Goal reached:  3299
Score:  10
 
Episode:  3521
Goal reached:  3300
Score:  -1
 
Episode:  3522
Goal reached:  3301
Score:  13
 
Episode:  3523
Goal reached:  3302
Score:  5
 
Epi

Episode:  3677
Goal reached:  3456
Score:  5
 
Episode:  3678
Goal reached:  3457
Score:  4
 
Episode:  3679
Goal reached:  3458
Score:  11
 
Episode:  3680
Goal reached:  3459
Score:  11
 
Episode:  3681
Goal reached:  3460
Score:  2
 
Episode:  3682
Goal reached:  3461
Score:  4
 
Episode:  3683
Goal reached:  3462
Score:  4
 
Episode:  3684
Goal reached:  3463
Score:  -4
 
Episode:  3685
Goal reached:  3464
Score:  7
 
Episode:  3686
Goal reached:  3465
Score:  4
 
Episode:  3687
Goal reached:  3466
Score:  10
 
Episode:  3688
Goal reached:  3467
Score:  9
 
Episode:  3689
Goal reached:  3468
Score:  6
 
Episode:  3690
Goal reached:  3469
Score:  3
 
Episode:  3691
Goal reached:  3470
Score:  6
 
Episode:  3692
Goal reached:  3471
Score:  3
 
Episode:  3693
Goal reached:  3472
Score:  8
 
Episode:  3694
Goal reached:  3473
Score:  9
 
Episode:  3695
Goal reached:  3474
Score:  4
 
Episode:  3696
Goal reached:  3475
Score:  12
 
Episode:  3697
Goal reached:  3476
Score:  11
 
Episode

Episode:  3850
Goal reached:  3629
Score:  5
 
Episode:  3851
Goal reached:  3630
Score:  9
 
Episode:  3852
Goal reached:  3631
Score:  10
 
Episode:  3853
Goal reached:  3632
Score:  8
 
Episode:  3854
Goal reached:  3633
Score:  -5
 
Episode:  3855
Goal reached:  3634
Score:  3
 
Episode:  3856
Goal reached:  3635
Score:  6
 
Episode:  3857
Goal reached:  3636
Score:  6
 
Episode:  3858
Goal reached:  3637
Score:  6
 
Episode:  3859
Goal reached:  3638
Score:  4
 
Episode:  3860
Goal reached:  3639
Score:  8
 
Episode:  3861
Goal reached:  3640
Score:  -6
 
Episode:  3862
Goal reached:  3641
Score:  -3
 
Episode:  3863
Goal reached:  3642
Score:  4
 
Episode:  3864
Goal reached:  3643
Score:  5
 
Episode:  3865
Goal reached:  3644
Score:  9
 
Episode:  3866
Goal reached:  3645
Score:  -5
 
Episode:  3867
Goal reached:  3646
Score:  5
 
Episode:  3868
Goal reached:  3647
Score:  9
 
Episode:  3869
Goal reached:  3648
Score:  12
 
Episode:  3870
Goal reached:  3649
Score:  -2
 
Episod

Episode:  4023
Goal reached:  3802
Score:  6
 
Episode:  4024
Goal reached:  3803
Score:  6
 
Episode:  4025
Goal reached:  3804
Score:  -6
 
Episode:  4026
Goal reached:  3805
Score:  8
 
Episode:  4027
Goal reached:  3806
Score:  12
 
Episode:  4028
Goal reached:  3807
Score:  -4
 
Episode:  4029
Goal reached:  3808
Score:  13
 
Episode:  4030
Goal reached:  3809
Score:  7
 
Episode:  4031
Goal reached:  3810
Score:  8
 
Episode:  4032
Goal reached:  3811
Score:  12
 
Episode:  4033
Goal reached:  3812
Score:  10
 
Episode:  4034
Goal reached:  3813
Score:  7
 
Episode:  4035
Goal reached:  3814
Score:  9
 
Episode:  4036
Goal reached:  3815
Score:  2
 


KeyboardInterrupt: 

To train the agent we use the function see in the step 4. 

We start having a pure exploration to fill the Q table but with the episodes increse we want that the explotation increse too. To obtain that we are decresing epsilon every episode. After 200 episode (20000 step) we will be in a pure explotation state.


Each episode require a different number of steps to complete the task, so we don’t always receive the same reward (each move is -1 point).

The minimum reward will be 3 (20 - 17), it takes 16 moves + 1 pick up action if we initialised the taxi and passenger at opposite corners with a drop-off location being the same corner as the taxi’s original position. 

The maximum reward is 15, if we take the two closest colour squares (Red and Yellow), the agent would use 1 pick up + 4 moves.

# Result

The program should run for hundreds/thousands of episodes (many hours) to learn and have good result, so I will attach some results.

At the beginning we have pure exploration, the agent will explore every situation making a lot of mistakes.

    Episode:  1
    Goal reached:  0
    Score:  -415
    
After 100 episodes we are in a 50/50 between exploration and explotation, due to the decreasing value of epsilon. The agent is still learning but it's possible to reach some goal with big negative score.

    Episode:  100
    Goal reached:  0
    Score:  -325

After 200 episodes we can see that the agent is still making mistakes but it start to reach some goals more frequently. Now, we are in pure explotation, the agent should have learned enough to take good decision.

    Episode:  200
    Goal reached:  21
    Score:  -118

We see that in the following next 100 episode (201 to 300) it reached the goal more often.

    Episode:  300
    Goal reached:  92
    Score:  -31

Then, after 400 episodes and more the agent knows perfectly how to interact in every situation taking the best action and the shortest way to reach the goal. It's rare that he doesn't reach the goal. 

    Episode:  401
    Goal reached:  182
    Score:  -2
    
    Episode:  4000
    Goal reached:  3779
    Score:  10